In [5]:
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium
# import all the libraries

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source)
table = soup.find('table', class_="wikitable sortable")
# find the table contains the Postal Code 
datalist = table.find_all('td')
# find all the data we needed in <td> 

In [7]:
df = pd.DataFrame()
x=0
while True:
    PostalCode = datalist[3*x].text
    Borough = datalist[3*x+1].text
    Neighborhood = datalist[3*x+2].text
    Neighborhood = Neighborhood.strip('\n')
    df = df.append(pd.DataFrame({'Postal Code': PostalCode, 'Borough': Borough, 'Neighborhood': Neighborhood}, index = [0]), ignore_index = True)
# use len(datalist) to get the length of the list then divide it into  three groups.
    if x == ((len(datalist)-3)/3):
        break
    x+=1
df = df[df['Borough'] != 'Not assigned']
# if the value of Neighborhood is Not assigned, then copy the value of borough into it
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [11]:
df_group = df.groupby(['Postal Code', 'Borough']).agg({'Neighborhood':','.join}).reset_index()
#df_group.head()

In [12]:
df_group.shape

(103, 3)

In [13]:
df_geo = pd.DataFrame()
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
# create a new DataFrame to contain all the data from Geospatial_Coordinates.csv

In [14]:
df_gp = df_group.merge(df_geo, how='left', on=['Postal Code'])
df_gp
# merge these two tables

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
